# 05 — Prescriptive Recommender System

**Amaç:** 04. notebook'ta eğitilen dürüst "Conversion Predictor" modelini kullanarak, her müşteri için **hangi Kampanya Kanalı + Platform** ikilisinin dönüşüm olasılığını en çok artıracağını öneren bir **Prescriptive (Reçeteleyici) Öneri Sistemi** kurmak.

**Akış:** 1) Varlık yükleme (model, scaler, imputer, test verisi) → 2) Strateji simülasyonu (`recommend_action`) → 3) Performans Lift analizi → 4) Segment–kanal heatmap ve feature importance ilişkisi.

## Predictive vs Prescriptive Analitik

- **Predictive (Tahmine dayalı):** "Ne olacak?" sorusuna cevap verir. Örneğin: Bu müşterinin mevcut kanal/platform ile dönüşüm yapma olasılığı nedir?
- **Prescriptive (Reçeteleyici):** "Ne yapmalıyız?" sorusuna cevap verir. Örneğin: Bu müşteriyi dönüştürmek için hangi kanal+platform kombinasyonunu kullanmalıyız?

Bu notebook'ta, aynı tahmin modelini **simülasyon** ile kullanıyoruz: Tüm olası aksiyonları (kanal+platform çiftleri) deneyip en yüksek dönüşüm olasılığını veren aksiyonu **öneri** olarak sunuyoruz. Yani model pasif bir tahminci olmaktan çıkıp **aktif bir strateji motoru**na dönüşüyor.

## Model Persistence: Neden .pkl Dosyaları?

04. notebook'ta eğitilen model, scaler ve imputer **disk üzerinde** `.pkl` (pickle) dosyaları olarak kaydedildi. Böylece:

1. **Yeniden eğitim gerekmez:** Aynı pipeline (özellik hazırlama → impute → scale → tahmin) başka bir ortamda (bu notebook, API, vb.) tekrarlanabilir.
2. **Versiyonlama:** Hangi modelin kullanıldığı nettir (`models/final_model.pkl`).
3. **Tutarlılık:** Scaler ve imputer, eğitim verisi üzerinde fit edildiği için test/öneri verisinde **aynı dönüşümler** uygulanmalıdır; bu dosyalar olmadan özellik uzayı uyumsuz kalır.

Bu notebook, `models/final_model.pkl`, `models/scaler.pkl` ve `models/imputer.pkl` dosyalarını yükleyerek 04. notebook ile **aynı** modeli kullanır.

## Klasik User-Based Collaborative Filtering'dan Farkı

- **User-Based CF:** "Senin gibi kullanıcılar şunu beğendi" der; benzer kullanıcıların geçmiş davranışına dayanır. Çıktı genelde **ürün/içerik önerisi** (rating/item).
- **Bu sistem:** Müşteri **profil özelliklerine** (yaş, gelir, geçmiş alışveriş vb.) ve **alabileceğimiz aksiyona** (hangi kanal+platform) bakarak, **dönüşüm olasılığını maksimize eden aksiyonu** önerir. Yani **aksiyon odaklı (action-oriented)** bir prescriptive sistemdir; "kime ne yapmalıyız?" sorusuna yanıt verir.

## 1. Asset Retrieval

Aşağıda 04. notebook ile uyumlu olacak şekilde:
- `models/final_model.pkl`, `scaler.pkl`, `imputer.pkl` yükleniyor.
- `marketing_analytics_featured.csv` üzerinden veri alınıyor; ROI_v2 ve CPA_v2 eklenip **Scenario D** feature seti ile `prepare_X` uygulanıyor.
- Aynı `random_state` ile train/test split yapılarak **test seti** elde ediliyor (lift analizi için).

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from sklearn.impute import SimpleImputer

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# --- Model, scaler, imputer yükleme (04. notebook ile aynı yol) ---
MODELS_DIR = "../models"
path_model = os.path.join(MODELS_DIR, "final_model.pkl")
path_scaler = os.path.join(MODELS_DIR, "scaler.pkl")
path_imputer = os.path.join(MODELS_DIR, "imputer.pkl")

assert os.path.exists(
    path_model), f"Model bulunamadı: {path_model}. Önce 04_model_comparison.ipynb çalıştırıp modeli kaydedin."
model = joblib.load(path_model)
scaler = joblib.load(path_scaler)
imputer = joblib.load(path_imputer)
print("Yüklendi: final_model, scaler, imputer")

Yüklendi: final_model, scaler, imputer


In [2]:
print("Yüklendi: final_model, scaler, imputer")
# --- Veri yükleme (04 ile aynı kaynak) ---
path_data = "../data/marketing_analytics_featured.csv"
df = pd.read_csv(path_data)
y = df["Conversion"]

Yüklendi: final_model, scaler, imputer


In [3]:
# 04. notebook'taki gibi ROI_v2 ve CPA_v2 ekle (Scenario E/D feature seti için)
df["ROI_v2"] = (df["Income"] * df["ClickThroughRate"]) / (df["AdSpend"].replace(0, np.nan).fillna(1) + 1)
df["CPA_v2"] = df["AdSpend"] / (df["WebsiteVisits"].replace(0, np.nan).fillna(1) + 1)

In [4]:
for c in ["ROI_v2", "CPA_v2"]:
    df[c] = df[c].replace([np.inf, -np.inf], np.nan)
    df[c] = df[c].clip(upper=df[c].quantile(0.995))

In [5]:
# Scenario D: 04 ile aynı drop listesi
BASE_DROP = ["CustomerID", "Conversion"]
SCENARIOS = {
    "D": BASE_DROP + ["ConversionRate", "CTR_to_Conversion", "ROI_Proxy", "CPA_Proxy"],
}


In [6]:
def prepare_X(data, drop_cols):
    """drop_cols içindeki sütunları çıkarır, kategorikleri one-hot encode eder (04 ile aynı)."""
    to_drop = [c for c in drop_cols if c in data.columns]
    X = data.drop(columns=to_drop, errors="ignore")
    X = pd.get_dummies(X, drop_first=True)
    for c in X.select_dtypes(include=["bool"]).columns:
        X[c] = X[c].astype(int)
    return X

In [7]:
X_all = prepare_X(df, SCENARIOS["D"])
feature_columns = X_all.columns.tolist()
print("Feature sayısı:", len(feature_columns))

Feature sayısı: 55


In [8]:
# --- Train/Test split (04 ile aynı; test seti lift analizi için) ---
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
# Test setindeki orijinal satır indeksleri (df üzerinden test müşterilerine erişim için)
train_idx, test_idx = train_test_split(
    np.arange(len(df)), test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
df_test = df.iloc[test_idx].reset_index(drop=True)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)
X_test_s = scaler.transform(X_test_imputed)
print("Test seti boyutu:", len(df_test))

Test seti boyutu: 9600
